import library

In [45]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Parameter, DataParallel
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
import shutil
import os
from PIL import Image
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [30]:
# TODO:
# 1. add batch size
# 2. SDG
# lr scheduler
# weight decay (optimizer)
# weight decay (lr scheduler)
# add resnet50
# change generator ****
# validation

# save to numpy
# data.npy
# label.npy
# data_1.npy(shape) 500000,112,112,3
# label_1.npy(shape) 500000,1

build dataset

In [35]:
def get_path_df(identity_root, identity_list):
    path_list = []
    identity_label_list = []
    for identity in tqdm(identity_list):
        image_list = os.listdir(os.path.join(identity_root, identity)) 
        for img in image_list:
            path = os.path.join(identity_root, identity, img)
            path_list.append(path)
            identity_label_list.append(identity) 

    data_df = pd.DataFrame({'img_path': path_list, 'identity': identity_label_list})
    data_df['identity_code'] = pd.Categorical(data_df['identity']).codes # convert identity to unique code(int)
    data_df['identity_code'] = data_df['identity_code'].astype('int32')
    return data_df

In [80]:
# buid the dataframe from the the path
identity_root = 'lfw_funneled'
identity_list = os.listdir(identity_root)
identity_list = [identity for identity in identity_list if os.path.isdir(os.path.join(identity_root, identity))] #only folder is identity

# train test val split
# train_identity_list, test_identity_list = train_test_split(identity_list, test_size=0.2, random_state=42)
# test_identity_list, val_identity_list = train_test_split(test_identity_list, test_size=0.5, random_state=42)

# build the dataframe
# train_df = get_path_df(identity_root, train_identity_list)
# test_df = get_path_df(identity_root, test_identity_list)
# val_df = get_path_df(identity_root, val_identity_list)
# print('train: {}, val: {}, test: {}'.format(len(train_df), len(val_df), len(test_df)))

# total data for test
path_df = get_path_df(identity_root, identity_list)

100%|██████████| 5749/5749 [00:00<00:00, 7835.52it/s]


In [58]:
# save numpy array from path
def get_img_npy(path_df, img_shape):
    img_npy = np.zeros((len(path_df), img_shape[0], img_shape[1], img_shape[2]), dtype=np.uint8)
    label_npy = np.zeros((len(path_df), 1), dtype=np.uint8)
    for i in tqdm(range(len(path_df))):
        img_path = path_df.iloc[i]['img_path']
        img = Image.open(img_path)
        img_npy[i] = np.array(img)
        label_npy[i] = path_df.iloc[i]['identity_code']
    return img_npy, label_npy 

In [83]:
df = path_df
img_shape = (250, 250, 3)
img_npy, label_npy = get_img_npy(df, img_shape)

100%|██████████| 13232/13232 [00:57<00:00, 229.44it/s]


In [97]:
class FaceDataset(Dataset):
    def __init__(self, img_npy_list, label_npy_list, input_shape, phase="train"):
        self.img_npy = np.vstack(img_npy_list)
        self.label_npy = np.vstack(label_npy_list)
        self.phase = phase
        self.input_shape = input_shape
        if self.phase == 'train':
            self.transforms = T.Compose([
                T.RandomCrop(self.input_shape[1:]),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize(mean=[0.5], std=[0.5])
            ])
        else:   
            self.transforms = T.Compose([
                T.CenterCrop(self.input_shape[1:]),
                T.ToTensor(),
                T.Normalize(mean=[0.5], std=[0.5])
            ])
            
    def __len__(self):
        return len(self.label_npy)
    
    def __getitem__(self, index):
        data = self.img_npy[index]
        label = self.label_npy[index]
        data = Image.fromarray(data)
        data = data.convert('L')
        data = self.transforms(data)
        return data.float(), label

In [98]:
# dataloader
img_npy_list = [img_npy]
label_npy_list = [label_npy]
input_shape = (1, 250, 250)
phase = 'train'
dataset = FaceDataset(img_npy_list, label_npy_list, input_shape, phase)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
num_classes = len(np.unique(path_df['identity'].values))
print(num_classes)

5749


In [99]:
for data, label in dataloader:
    print(data.shape)
    print(label.shape)
    break

torch.Size([32, 1, 250, 250])
torch.Size([32, 1])


build model

In [25]:
# load pretrained model
model_resnet50 = resnet50(pretrained=False) 
# change first and last layer
model_resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model_resnet50.fc = nn.Linear(512, 512)

loss function

In [26]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin

            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output

train

In [28]:
# test the training step
data_df = val_df.copy()    
num_classes = len(np.unique(data_df['identity'].values))
model = DataParallel(model_resnet50)
metric_fc = DataParallel(ArcMarginProduct(512, num_classes, s=30, m=0.5, easy_margin=False))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}], lr=0.001)
model = model.train()

In [11]:
for data, label in tqdm(dataloader):
    feature = model(data)
    output = metric_fc(feature, label)
    loss = criterion(output, label.long().cuda())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    output = output.data.cpu().numpy()
    output = np.argmax(output, axis=1)
    label = label.data.cpu().numpy()
    acc = np.mean((output == label).astype(int))

100%|██████████| 48/48 [00:08<00:00,  5.37it/s]


In [15]:
# hyper parameters
data_df = data_df.copy()
num_epochs = 2
batch_size = 32
learning_rate = 0.001
num_classes = len(np.unique(data_df['identity'].values))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
classes = data_df['identity_code'].unique()

# model, metric_fc, criterion, optimizer
model = DataParallel(model_resnet50)
metric_fc = DataParallel(ArcMarginProduct(512, num_classes, s=30, m=0.5, easy_margin=False))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}], lr=learning_rate)
writer = SummaryWriter(log_dir='log/v2')

In [ ]:
# train
model.train()
step = 0
for epoch in range(num_epochs):
    for data, label in tqdm(dataloader, desc='Epoch {}/{}'.format(epoch+1, num_epochs)):
        # forward
        data = data.to(device)
        label = label.to(device)
        feature = model(data).to(device)
        output = metric_fc(feature, label).to(device)
        loss = criterion(output, label.long().cuda())

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # visualize
        img_grid = make_grid(data)
        img = data.cpu().numpy()
        img = img.reshape(img.shape[0], -1)
        output = output.data.cpu().numpy()
        output = np.argmax(output, axis=1)
        label = label.data.cpu().numpy()
        acc = np.mean((output == label).astype(int))

        # writer.add_histogram('fc', model_resnet18.fc.weight, epoch)
        writer.add_image('image', img_grid, step)
        writer.add_scalar('loss', loss.item(), step)
        writer.add_scalar('acc', acc, step)
        writer.add_embedding(img, metadata=label, label_img=data, global_step=step)
        step += 1

In [2]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile